# OCSVM rbf

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Libraries import

In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_recall_curve
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from scipy.spatial import distance
import time
from sklearn.svm import OneClassSVM

## Prediction translation

In [4]:
def def_outlier(df):
    if (df['y_pred'] in [-1]):
        val = 1
    else:
        val = 0
    return val

## Arrhythmia

**Dataset source**: http://odds.cs.stonybrook.edu/arrhythmia-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources**:

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

F. Keller, E. Muller, K. Bohm.“HiCS: High-contrast subspaces for density-based outlier ranking.” ICDE, 2012.

In [5]:
data = pd.read_csv('./arrhythmia.csv', sep = ',')

In [6]:
# dropping columns that consist only of 0's
data = data.drop(columns = ['Col15', 'Col63', 'Col65', 'Col79', 'Col127', 'Col128','Col135', 'Col137', 'Col139','Col141',
'Col147', 'Col152', 'Col153','Col160','Col200', 'Col260', 'Col270'])

In [7]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col265,Col266,Col267,Col268,Col269,Col271,Col272,Col273,Col274,y
0,75.0,0.0,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,-0.3,0.0,9.0,-0.9,0.0,0.9,2.9,23.3,49.4,1
1,56.0,1.0,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,-0.5,0.0,8.5,0.0,0.0,0.2,2.1,20.4,38.8,0
2,54.0,0.0,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.9,0.0,9.5,-2.4,0.0,0.3,3.4,12.3,49.0,0
3,55.0,0.0,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.1,0.0,12.2,-2.2,0.0,0.4,2.6,34.6,61.6,0
4,75.0,0.0,190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,-0.4,0.0,13.1,-3.6,0.0,-0.1,3.9,25.4,62.8,1


In [8]:
data.shape

(452, 258)

In [9]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,386
1,66


### One-Class SVM rbf

In [10]:
features = data.drop(columns = ['y'])
target = data['y']

In [11]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [12]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
arrhythmia_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()
yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
arrhythmia_rbf_oc_svm_test_time = end - start
print(end - start)

0.015625
0.015625


In [13]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [14]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
arrhythmia_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.7626000942063118

In [15]:
arrhythmia_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92       386
           1       0.49      0.35      0.41        66

    accuracy                           0.85       452
   macro avg       0.69      0.64      0.66       452
weighted avg       0.83      0.85      0.84       452



In [16]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
arrhythmia_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(arrhythmia_ocsvm_rbf_auc_precision_recall)

0.4089494760599974


## Cardiotocography

**Dataset source**: http://odds.cs.stonybrook.edu/cardiotocogrpahy-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [17]:
data = pd.read_csv('./Cardiotocography.csv')

In [18]:
data.shape

(1831, 22)

In [19]:
data['y'] = data['y'].astype(int)

In [20]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,1655
1,176


In [21]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,y
0,0.004912,0.693191,-0.203640,0.595322,0.353190,-0.061401,-0.278295,-1.650444,0.759072,-0.420487,...,-0.798376,1.854728,0.622631,0.963083,0.301464,0.193113,0.231498,-0.289786,-0.493294,0
1,0.110729,-0.079903,-0.203640,1.268942,0.396246,-0.061401,-0.278295,-1.710270,0.759072,-0.420487,...,-0.798376,1.854728,0.278625,0.963083,0.301464,0.129265,0.093563,-0.256385,-0.493294,0
2,0.216546,-0.272445,-0.203640,1.050988,0.148753,-0.061401,-0.278295,-1.710270,1.106509,-0.420487,...,-1.332931,0.314688,2.342663,-0.488279,0.061002,0.065417,0.024596,-0.256385,1.140018,0
3,0.004912,0.727346,-0.203640,1.212171,-0.683598,-0.061401,-0.278295,-1.710270,1.106509,-0.420487,...,-1.332931,0.314688,1.654650,-0.488279,0.061002,0.193113,0.093563,-0.323186,1.140018,0
4,-0.100905,0.363595,1.321366,1.027120,0.141359,-0.061401,-0.278295,-0.992364,-0.051613,-0.420487,...,-0.085638,-0.565334,0.278625,-0.488279,-0.059229,0.065417,0.024596,-0.456787,1.140018,0


### One-Class SVM rbf

In [22]:
features = data.drop(columns = ['y'])
target = data['y']

In [23]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [24]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
cardio_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
cardio_rbf_oc_svm_test_time = end - start
print(end - start)

0.03125
0.0625


In [25]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [26]:
confusion_matrix(target, predictions)

array([[1547,  108],
       [ 100,   76]], dtype=int64)

In [27]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
cardio_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.856052595440813

In [28]:
cardio_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      1655
           1       0.41      0.43      0.42       176

    accuracy                           0.89      1831
   macro avg       0.68      0.68      0.68      1831
weighted avg       0.89      0.89      0.89      1831



In [29]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
cardio_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(cardio_ocsvm_rbf_auc_precision_recall)

0.41334053839828533


## ForestCover

**Dataset source**: http://odds.cs.stonybrook.edu/forestcovercovertype-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

Kai Ming Ting, Guang-Tong Zhou, Fei Tony Liu & Tan Swee Chuan. (2010). Mass Estimation and Its Applications. Proceedings of The 16th ACM SIGKDD Conference on Knowledge Discovery and Data Mining 2010. pp. 989-998.

Swee Chuan Tan, Kai Ming Ting & Fei Tony Liu. (2011). Fast Anomaly Detection for Streaming Data. Proceedings of the International Joint Conference on Artificial Intelligence 2011. pp.1151-1156.

In [30]:
data = pd.read_csv('./ForestCover.csv')

In [31]:
data.shape

(286048, 11)

In [32]:
pd.pivot_table(data,
             values = 'Col2',
               index = 'y', 
              aggfunc = 'count')

,Col2
y,
0,283301
1,2747


In [33]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,y
0,2804,139,9,268,65,3180,234,238,135,6121,0
1,2785,155,18,242,118,3090,238,238,122,6211,0
2,2579,132,6,300,-15,67,230,237,140,6031,0
3,2886,151,11,371,26,5253,234,240,136,4051,0
4,2742,134,22,150,69,3215,248,224,92,6091,0


### One-Class SVM rbf

In [34]:
features = data.drop(columns = ['y'])
target = data['y']

In [35]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [36]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
forestcover_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
forestcover_rbf_oc_svm_test_time = end - start
print(end - start)

1130.015625
1677.4375


In [37]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [38]:
confusion_matrix(target, predictions)

array([[256805,  26496],
       [   637,   2110]], dtype=int64)

In [39]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
forestcover_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.9312673091740444

In [40]:
forestcover_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95    283301
           1       0.07      0.77      0.13      2747

    accuracy                           0.91    286048
   macro avg       0.54      0.84      0.54    286048
weighted avg       0.99      0.91      0.94    286048



In [41]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
forestcover_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(forestcover_ocsvm_rbf_auc_precision_recall)

0.12859142599282802


## Annthyroid

**Dataset source**: http://odds.cs.stonybrook.edu/annthyroid-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Aditional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

In [42]:
data = pd.read_csv('./annthyroid.csv')

In [43]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,y
0,0.73,0.00060,0.015,0.120,0.082,0.146,0
1,0.24,0.00025,0.030,0.143,0.133,0.108,0
2,0.47,0.00190,0.024,0.102,0.131,0.078,0
3,0.64,0.00090,0.017,0.077,0.090,0.085,0
4,0.23,0.00025,0.026,0.139,0.090,0.153,0


In [44]:
data.shape

(7200, 7)

In [45]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,6666
1,534


### One-Class SVM rbf

In [46]:
features = data.drop(columns = ['y'])
target = data['y']

In [47]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [48]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
annthyroid_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
annthyroid_rbf_oc_svm_test_time = end - start
print(end - start)

0.4375
1.046875


In [49]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [50]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
annthyroid_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.61818934702459

In [51]:
annthyroid_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      6666
           1       0.17      0.23      0.19       534

    accuracy                           0.86      7200
   macro avg       0.55      0.57      0.56      7200
weighted avg       0.88      0.86      0.87      7200



In [52]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
annthyroid_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(annthyroid_ocsvm_rbf_auc_precision_recall)

0.1440984150992436


## Credit card

**Dataset source**: https://www.kaggle.com/mlg-ulb/creditcardfraud

**Additional sources:**

Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon

Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE

Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)

Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier

Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing

Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Deep-Learning Domain Adaptation Techniques for Credit Cards Fraud Detection, INNSBDDL 2019: Recent Advances in Big Data and Deep Learning, pp 78-88, 2019

Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combining Unsupervised and Supervised Learning in Credit Card Fraud Detection Information Sciences, 2019

Yann-Aël Le Borgne, Gianluca Bontempi Machine Learning for Credit Card Fraud Detection - Practical Handbook

In [53]:
data = pd.read_csv('./creditcard.csv')

In [54]:
data = data.drop(columns = ['Time'])

In [55]:
data.shape

(284807, 30)

In [56]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [57]:
pd.pivot_table(data,
             values = 'V1',
               index = 'Class', 
              aggfunc = 'count')

,V1
Class,
0,284315
1,492


### One-Class SVM rbf

In [58]:
features = data.drop(columns = ['Class'])
target = data['Class']

In [59]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [60]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
creditcard_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
creditcard_rbf_oc_svm_test_time = end - start
print(end - start)

1682.75
1827.734375


In [61]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [62]:
accuracy_score(predictions, target)

0.9013121166263469

In [63]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
creditcard_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.9454014419767152

In [64]:
creditcard_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95    284315
           1       0.02      0.88      0.03       492

    accuracy                           0.90    284807
   macro avg       0.51      0.89      0.49    284807
weighted avg       1.00      0.90      0.95    284807



In [65]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
creditcard_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(creditcard_ocsvm_rbf_auc_precision_recall)

0.38668923406828387


## Mammography

**Dataset source**: http://odds.cs.stonybrook.edu/mammography-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

In [66]:
data = pd.read_csv('./mammography.csv')

In [67]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,y
0,0.230020,5.072578,-0.276061,0.832444,-0.377866,0.480322,0
1,0.155491,-0.169390,0.670652,-0.859553,-0.377866,-0.945723,0
2,-0.784415,-0.443654,5.674705,-0.859553,-0.377866,-0.945723,0
3,0.546088,0.131415,-0.456387,-0.859553,-0.377866,-0.945723,0
4,-0.102987,-0.394994,-0.140816,0.979703,-0.377866,1.013566,0


In [68]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,10923
1,260


In [69]:
data.shape

(11183, 7)

### One-Class SVM rbf

In [70]:
features = data.drop(columns = ['y'])
target = data['y']

In [71]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [72]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
mammography_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
mammography_rbf_oc_svm_test_time = end - start
print(end - start)

1.203125
2.53125


In [73]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [74]:
confusion_matrix(target, predictions)

array([[10042,   881],
       [  147,   113]], dtype=int64)

In [75]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
mammography_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.6349551757406743

In [76]:
mammography_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.99      0.92      0.95     10923
           1       0.11      0.43      0.18       260

    accuracy                           0.91     11183
   macro avg       0.55      0.68      0.57     11183
weighted avg       0.97      0.91      0.93     11183



In [77]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
mammography_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(mammography_ocsvm_rbf_auc_precision_recall)

0.11314186844336357


## Shuttle

**Dataset source**: http://odds.cs.stonybrook.edu/shuttle-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

Kai Ming Ting, Guang-Tong Zhou, Fei Tony Liu & Tan Swee Chuan. (2010). Mass Estimation and Its Applications. Proceedings of The 16th ACM SIGKDD Conference on Knowledge Discovery and Data Mining 2010. pp. 989-998.

Swee Chuan Tan, Kai Ming Ting & Fei Tony Liu. (2011). Fast Anomaly Detection for Streaming Data. Proceedings of the International Joint Conference on Artificial Intelligence 2011. pp.1151-1156.

In [78]:
data = pd.read_csv('./shuttle.csv', sep = ',')

In [79]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,y
0,50,21,77,0,28,0,27,48,22,1
1,53,0,82,0,52,-5,29,30,2,0
2,37,0,76,0,28,18,40,48,8,0
3,37,0,79,0,34,-26,43,46,2,0
4,85,0,88,-4,6,1,3,83,80,1


In [80]:
data.shape

(49097, 10)

In [81]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,45586
1,3511


### One-Class SVM rbf

In [82]:
features = data.drop(columns = ['y'])
target = data['y']

In [83]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [84]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
shuttle_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
shuttle_rbf_oc_svm_test_time = end - start
print(end - start)

31.578125
47.859375


In [85]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [86]:
confusion_matrix(target, predictions)

array([[42981,  2605],
       [ 1214,  2297]], dtype=int64)

In [87]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
shuttle_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.9434906043235352

In [88]:
shuttle_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96     45586
           1       0.47      0.65      0.55      3511

    accuracy                           0.92     49097
   macro avg       0.72      0.80      0.75     49097
weighted avg       0.94      0.92      0.93     49097



In [89]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
shuttle_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(shuttle_ocsvm_rbf_auc_precision_recall)

0.6885763617839639


## mnist

**Dataset source**: http://odds.cs.stonybrook.edu/mnist-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Bandaragoda, Tharindu R., et al. “Efficient Anomaly Detection by Isolation Using Nearest Neighbour Ensemble.” 2014 IEEE International Conference on Data Mining Workshop. IEEE, 2014.

In [90]:
data = pd.read_csv('./mnist.csv')

In [91]:
data = data.drop(columns = ['Col1','Col4', 'Col7', 'Col22', 'Col27', 'Col29', 'Col38', 'Col41', 'Col51', 'Col53', 'Col54', 'Col61', 'Col62', 'Col71', 'Col73', 'Col79', 'Col87', 'Col88', 'Col89', 'Col90',
'Col92', 'Col100'])

In [92]:
pd.pivot_table(data,
             values = 'Col2',
               index = 'y', 
              aggfunc = 'count')

,Col2
y,
0,6903
1,700


In [93]:
data.shape

(7603, 79)

In [94]:
data.head()

,Col2,Col3,Col5,Col6,Col8,Col9,Col10,Col11,Col12,Col13,...,Col86,Col91,Col93,Col94,Col95,Col96,Col97,Col98,Col99,y
0,-73.804153,198.205963,-13.124617,-1.1501,-0.141633,179.249390,114.661163,-80.736702,130.659348,162.649841,...,-15.392716,188.055649,-4.469967,158.381409,-137.100632,27.131416,-2.274633,-0.00065,-12.351267,0
1,-73.804153,197.205963,-13.124617,-1.1501,-0.141633,179.249390,-44.338833,-80.736702,128.659348,190.649841,...,-15.392716,186.055649,-4.469967,123.381416,-137.100632,157.131409,-2.274633,-0.00065,-12.351267,0
2,-73.804153,-53.794033,-13.124617,-1.1501,-0.141633,-73.750618,-44.338833,170.263306,130.659348,46.649849,...,-15.392716,188.055649,-4.469967,157.381409,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0
3,-73.804153,86.205963,-13.124617,-1.1501,-0.141633,76.249382,208.661163,107.263298,130.659348,190.649841,...,-15.392716,188.055649,-4.469967,157.381409,-137.100632,74.131416,-2.274633,-0.00065,-12.351267,0
4,-27.804153,199.205963,-13.124617,-1.1501,-0.141633,179.249390,-44.338833,-80.736702,130.659348,91.649849,...,-15.392716,188.055649,-4.469967,22.381416,-137.100632,159.131409,-2.274633,-0.00065,-12.351267,0


### One-Class SVM rbf

In [95]:
features = data.drop(columns = ['y'])
target = data['y']

In [96]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [97]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
mnist_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
mnist_rbf_oc_svm_test_time = end - start
print(end - start)

0.671875
1.5625


In [98]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [99]:
confusion_matrix(target, predictions)

array([[6368,  535],
       [ 470,  230]], dtype=int64)

In [100]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
mnist_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.7594308892613977

In [101]:
mnist_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      6903
           1       0.30      0.33      0.31       700

    accuracy                           0.87      7603
   macro avg       0.62      0.63      0.62      7603
weighted avg       0.87      0.87      0.87      7603



In [102]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
mnist_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(mnist_ocsvm_rbf_auc_precision_recall)

0.2613066133738081


## vowels

**Dataset source**: http://odds.cs.stonybrook.edu/japanese-vowels-data/

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [103]:
data = pd.read_csv('./vowels.csv')

In [104]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0.0,1406
1.0,50


In [105]:
data.shape

(1456, 13)

In [106]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,y
0,0.580469,-0.902534,0.617899,-0.997942,-2.463799,-0.846455,2.349849,0.375400,-0.649334,1.604637,-0.623060,-0.383125,0.0
1,0.784375,-1.077366,0.615781,-0.921911,-2.388553,-0.638047,2.106684,0.361018,-0.714317,1.260236,-0.423339,-0.287791,0.0
2,0.791292,-1.086242,0.669773,-0.806112,-2.260781,-0.538491,2.053282,0.266492,-0.842815,1.081797,-0.267201,-0.172203,0.0
3,1.217306,-1.083425,0.855483,-0.724879,-2.155552,-0.101879,1.768597,0.303151,-1.044710,0.655290,0.214298,-0.341840,0.0
4,1.065352,-1.030178,0.773297,-0.452289,-1.955907,0.248205,1.530474,0.253740,-0.968961,-0.208287,0.331578,0.007288,0.0


### One-Class SVM rbf

In [107]:
features = data.drop(columns = ['y'])
target = data['y']

In [108]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [109]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
vowels_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
vowels_rbf_oc_svm_test_time = end - start
print(end - start)

0.015625
0.046875


In [110]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [111]:
confusion_matrix(target, predictions)

array([[1278,  128],
       [  34,   16]], dtype=int64)

In [112]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
vowels_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.704651493598862

In [113]:
vowels_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.97      0.91      0.94      1406
           1       0.11      0.32      0.16        50

    accuracy                           0.89      1456
   macro avg       0.54      0.61      0.55      1456
weighted avg       0.94      0.89      0.91      1456



In [114]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
vowels_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(vowels_ocsvm_rbf_auc_precision_recall)

0.1643184998493822


## Seismic

**Dataset source**: http://odds.cs.stonybrook.edu/seismic-dataset/ (data is transformed from .arff to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [115]:
data = pd.read_csv('./seismic.csv', sep = ',')

In [116]:
data = data.drop(columns = ['nbumps6','nbumps7','nbumps89'])

In [117]:
data.shape

(2584, 16)

In [118]:
drop_enc = OneHotEncoder(drop='first').fit_transform(data[['seismic','seismoacoustic','shift','ghazard']])

In [119]:
cat_var = pd.DataFrame(drop_enc.toarray())
cat_var.columns = ['seismic: b', 'seismoacoustic: b','seismoacoustic: c','shift: W','ghazard: b','ghazard: c']

In [120]:
data = pd.concat([data, cat_var], axis = 1)

In [121]:
data = data.drop(columns = ['seismic','seismoacoustic','shift','ghazard'])

In [122]:
data.head()

,genergy,gpuls,gdenergy,gdpuls,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,energy,maxenergy,class,seismic: b,seismoacoustic: b,seismoacoustic: c,shift: W,ghazard: b,ghazard: c
0,15180,48,-72,-72,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,14720,33,-70,-79,1,0,1,0,0,2000,2000,0,0.0,0.0,0.0,0.0,0.0,0.0
2,8050,30,-81,-78,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,28820,171,-23,40,1,0,1,0,0,3000,3000,0,0.0,0.0,0.0,0.0,0.0,0.0
4,12640,57,-63,-52,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
pd.pivot_table(data,
             values = 'genergy',
               index = 'class', 
              aggfunc = 'count')

,genergy
class,
0,2414
1,170


### One-Class SVM rbf

In [124]:
features = data.drop(columns = ['class'])
target = data['class']

In [125]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [126]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
seismic_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
seismic_rbf_oc_svm_test_time = end - start
print(end - start)

0.0625
0.140625


In [127]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [128]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
seismic_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.4039621813928554

In [129]:
seismic_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      2414
           1       0.12      0.19      0.15       170

    accuracy                           0.86      2584
   macro avg       0.53      0.55      0.54      2584
weighted avg       0.89      0.86      0.87      2584



In [130]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
seismic_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(seismic_ocsvm_rbf_auc_precision_recall)

0.06450195977831653


## Musk

**Dataset source**: http://odds.cs.stonybrook.edu/musk-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

In [131]:
data = pd.read_csv('./musk.csv', sep = ',')

In [132]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col158,Col159,Col160,Col161,Col162,Col163,Col164,Col165,Col166,y
0,46.0,-108.0,-60.0,-69.0,-117.0,49.0,38.0,-161.0,-8.0,5.0,...,-308.0,52.0,-7.0,39.0,126.0,156.0,-50.0,-112.0,96.0,1.0
1,41.0,-188.0,-145.0,22.0,-117.0,-6.0,57.0,-171.0,-39.0,-100.0,...,-59.0,-2.0,52.0,103.0,136.0,169.0,-61.0,-136.0,79.0,1.0
2,46.0,-194.0,-145.0,28.0,-117.0,73.0,57.0,-168.0,-39.0,-22.0,...,-134.0,-154.0,57.0,143.0,142.0,165.0,-67.0,-145.0,39.0,1.0
3,41.0,-188.0,-145.0,22.0,-117.0,-7.0,57.0,-170.0,-39.0,-99.0,...,-60.0,-4.0,52.0,104.0,136.0,168.0,-60.0,-135.0,80.0,1.0
4,41.0,-188.0,-145.0,22.0,-117.0,-7.0,57.0,-170.0,-39.0,-99.0,...,-60.0,-4.0,52.0,104.0,137.0,168.0,-60.0,-135.0,80.0,1.0


In [133]:
data['y'] = data['y'].astype(int)

In [134]:
data.shape

(3062, 167)

In [135]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,2965
1,97


### One-Class SVM rbf

In [136]:
features = data.drop(columns = ['y'])
target = data['y']

In [137]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [138]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
musk_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
musk_rbf_oc_svm_test_time = end - start
print(end - start)

0.140625
0.375


In [139]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [140]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
musk_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.8548599641869925

In [141]:
musk_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.98      0.91      0.95      2965
           1       0.18      0.57      0.27        97

    accuracy                           0.90      3062
   macro avg       0.58      0.74      0.61      3062
weighted avg       0.96      0.90      0.93      3062



In [142]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
musk_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(musk_ocsvm_rbf_auc_precision_recall)

0.42561233559646655


## bank

**Dataset source**: https://github.com/GuansongPang/ADRepository-Anomaly-detection-datasets/tree/main/categorical%20data

Pang, G., Shen, C., Cao, L., & Hengel, A. V. D. (2021). Deep learning for anomaly detection: A review. ACM Computing Surveys (CSUR), 54(2), 1-38.

In [143]:
data = pd.read_csv('./bank.csv')

In [144]:
data.head()

,age,job=housemaid,job=services,job=admin.,job=blue-collar,job=technician,job=retired,job=management,job=unemployed,job=self-employed,...,previous,poutcome=nonexistent,poutcome=failure,poutcome=success,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,class
0,0.209877,0,0,0,0,0,0,0,0,0,...,0.000000,1,0,0,1.000000,0.882307,0.376569,0.980730,1.000000,0
1,0.296296,0,0,1,0,0,0,0,0,0,...,0.000000,1,0,0,1.000000,0.484412,0.615063,0.981183,1.000000,0
2,0.246914,1,0,0,0,0,0,0,0,0,...,0.000000,1,0,0,0.937500,0.698753,0.602510,0.957379,0.859735,0
3,0.160494,0,1,0,0,0,0,0,0,0,...,0.142857,0,1,0,0.333333,0.269680,0.192469,0.150759,0.512287,0
4,0.530864,0,0,0,1,0,0,0,0,0,...,0.000000,1,0,0,0.333333,0.340608,0.154812,0.174790,0.512287,1


In [145]:
pd.pivot_table(data,
             values = 'age',
               index = 'class', 
              aggfunc = 'count')

,age
class,
0,36548
1,4640


### One-Class SVM rbf

In [146]:
features = data.drop(columns = ['class'])
target = data['class']

In [147]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [148]:
start = time.process_time()

model = OneClassSVM(gamma='scale', nu=0.1)
model.fit(x_train_scaled)

end = time.process_time()
bank_rbf_oc_svm_train_time = end - start
print(end - start)

start = time.process_time()

yhat = model.predict(x_train_scaled)
y_scores = -model.decision_function(x_train_scaled)

end = time.process_time()
bank_rbf_oc_svm_test_time = end - start
print(end - start)

38.40625
41.515625


In [149]:
predictions = pd.DataFrame(yhat, columns = ['y_pred']).apply(def_outlier, axis = 1)

In [150]:
fpr, tpr, _ = metrics.roc_curve(target, y_scores)
bank_rbf_oc_svm_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.5705781874474003

In [151]:
bank_rbf_oc_svm_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91     36548
           1       0.26      0.23      0.25      4640

    accuracy                           0.84     41188
   macro avg       0.58      0.57      0.58     41188
weighted avg       0.83      0.84      0.84     41188



In [152]:
precision, recall, thresholds = precision_recall_curve(target, y_scores)
bank_ocsvm_rbf_auc_precision_recall = metrics.auc(recall, precision)
print(bank_ocsvm_rbf_auc_precision_recall)

0.19921512626272206


## Performance

In [153]:
performance = pd.DataFrame(columns = ['F1 score', 'recall', 'precision', 'AUC', 'AUPRC', 
                                      'Training time','Inference time','Total time'])

In [154]:
f1_score_oc_svm_rbf = {'arrhythmia':arrhythmia_rbf_oc_svm_report['1']['f1-score'],
                       'cardio':cardio_rbf_oc_svm_report['1']['f1-score'], 
                        'forestcover':forestcover_rbf_oc_svm_report['1']['f1-score'], 
                       'annthyroid':annthyroid_rbf_oc_svm_report['1']['f1-score'],       
                        'creditcard':creditcard_rbf_oc_svm_report['1']['f1-score'], 
                       'mammography':mammography_rbf_oc_svm_report['1']['f1-score'], 
                        'shuttle':shuttle_rbf_oc_svm_report['1']['f1-score'], 
                      'mnist':mnist_rbf_oc_svm_report['1']['f1-score'], 
                  'vowels':vowels_rbf_oc_svm_report['1']['f1-score'], 
                  'seismic':seismic_rbf_oc_svm_report['1']['f1-score'], 
                  'musk':musk_rbf_oc_svm_report['1']['f1-score'], 
                  'bank':bank_rbf_oc_svm_report['1']['f1-score']}
f1_score_oc_svm_rbf_df = pd.DataFrame.from_dict(f1_score_oc_svm_rbf, orient='index', columns = ['F1 score']).reset_index()

In [155]:
recall_oc_svm_rbf = {'arrhythmia':arrhythmia_rbf_oc_svm_report['1']['recall'],
                       'cardio':cardio_rbf_oc_svm_report['1']['recall'], 
                        'forestcover':forestcover_rbf_oc_svm_report['1']['recall'], 
                       'annthyroid':annthyroid_rbf_oc_svm_report['1']['recall'],       
                        'creditcard':creditcard_rbf_oc_svm_report['1']['recall'], 
                       'mammography':mammography_rbf_oc_svm_report['1']['recall'], 
                        'shuttle':shuttle_rbf_oc_svm_report['1']['recall'], 
                      'mnist':mnist_rbf_oc_svm_report['1']['recall'], 
                  'vowels':vowels_rbf_oc_svm_report['1']['recall'], 
                  'seismic':seismic_rbf_oc_svm_report['1']['recall'], 
                  'musk':musk_rbf_oc_svm_report['1']['recall'], 
                  'bank':bank_rbf_oc_svm_report['1']['recall'], }
recall_oc_svm_rbf_df = pd.DataFrame.from_dict(recall_oc_svm_rbf, orient='index', columns = ['Recall']).reset_index()

In [156]:
precision_oc_svm_rbf = {'arrhythmia':arrhythmia_rbf_oc_svm_report['1']['precision'],
                       'cardio':cardio_rbf_oc_svm_report['1']['precision'], 
                        'forestcover':forestcover_rbf_oc_svm_report['1']['precision'], 
                       'annthyroid':annthyroid_rbf_oc_svm_report['1']['precision'],       
                        'creditcard':creditcard_rbf_oc_svm_report['1']['precision'], 
                       'mammography':mammography_rbf_oc_svm_report['1']['precision'], 
                        'shuttle':shuttle_rbf_oc_svm_report['1']['precision'], 
                      'mnist':mnist_rbf_oc_svm_report['1']['precision'], 
                  'vowels':vowels_rbf_oc_svm_report['1']['precision'], 
                  'seismic':seismic_rbf_oc_svm_report['1']['precision'], 
                  'musk':musk_rbf_oc_svm_report['1']['precision'], 
                  'bank':bank_rbf_oc_svm_report['1']['precision'], }
precision_oc_svm_rbf_df = pd.DataFrame.from_dict(precision_oc_svm_rbf, orient='index', columns = ['Precision']).reset_index()

In [157]:
auc_oc_svm_rbf = {'arrhythmia':arrhythmia_rbf_oc_svm_auc,
                       'cardio':cardio_rbf_oc_svm_auc, 
                        'forestcover':forestcover_rbf_oc_svm_auc, 
                       'annthyroid':annthyroid_rbf_oc_svm_auc,       
                        'creditcard':creditcard_rbf_oc_svm_auc, 
                       'mammography':mammography_rbf_oc_svm_auc, 
                        'shuttle':shuttle_rbf_oc_svm_auc, 
                      'mnist':mnist_rbf_oc_svm_auc, 
                  'vowels':vowels_rbf_oc_svm_auc, 
                  'seismic':seismic_rbf_oc_svm_auc, 
                  'musk':musk_rbf_oc_svm_auc, 
                  'bank':bank_rbf_oc_svm_auc}
auc_oc_svm_rbf_df = pd.DataFrame.from_dict(auc_oc_svm_rbf, orient='index', columns = ['AUC']).reset_index()

In [158]:
auprc_oc_svm_rbf = {'arrhythmia':arrhythmia_ocsvm_rbf_auc_precision_recall,
                       'cardio':cardio_ocsvm_rbf_auc_precision_recall, 
                        'forestcover':forestcover_ocsvm_rbf_auc_precision_recall, 
                       'annthyroid':annthyroid_ocsvm_rbf_auc_precision_recall,       
                        'creditcard':creditcard_ocsvm_rbf_auc_precision_recall, 
                       'mammography':mammography_ocsvm_rbf_auc_precision_recall, 
                        'shuttle':shuttle_ocsvm_rbf_auc_precision_recall, 
                      'mnist':mnist_ocsvm_rbf_auc_precision_recall, 
                  'vowels':vowels_ocsvm_rbf_auc_precision_recall, 
                  'seismic':seismic_ocsvm_rbf_auc_precision_recall, 
                  'musk':musk_ocsvm_rbf_auc_precision_recall, 
                  'bank':bank_ocsvm_rbf_auc_precision_recall}
auprc_oc_svm_rbf_df = pd.DataFrame.from_dict(auprc_oc_svm_rbf, orient='index', columns = ['AUPRC']).reset_index()

In [159]:
training_time_oc_svm_rbf = {'arrhythmia':arrhythmia_rbf_oc_svm_train_time,
                       'cardio':cardio_rbf_oc_svm_train_time, 
                        'forestcover':forestcover_rbf_oc_svm_train_time, 
                       'annthyroid':annthyroid_rbf_oc_svm_train_time,       
                        'creditcard': creditcard_rbf_oc_svm_train_time, 
                       'mammography':mammography_rbf_oc_svm_train_time, 
                        'shuttle':shuttle_rbf_oc_svm_train_time, 
                      'mnist':mnist_rbf_oc_svm_train_time, 
                  'vowels':vowels_rbf_oc_svm_train_time, 
                  'seismic':seismic_rbf_oc_svm_train_time, 
                  'musk':musk_rbf_oc_svm_train_time, 
                  'bank':bank_rbf_oc_svm_train_time}
training_time_oc_svm_rbf_df = pd.DataFrame.from_dict(training_time_oc_svm_rbf, orient='index', columns = ['Training time']).reset_index()

In [160]:
test_time_oc_svm_rbf = {'arrhythmia':arrhythmia_rbf_oc_svm_test_time,
                       'cardio':cardio_rbf_oc_svm_test_time, 
                        'forestcover':forestcover_rbf_oc_svm_test_time, 
                       'annthyroid':annthyroid_rbf_oc_svm_test_time,       
                        'creditcard':creditcard_rbf_oc_svm_test_time, 
                       'mammography':mammography_rbf_oc_svm_test_time, 
                        'shuttle':shuttle_rbf_oc_svm_test_time, 
                      'mnist':mnist_rbf_oc_svm_test_time, 
                  'vowels':vowels_rbf_oc_svm_test_time, 
                  'seismic':seismic_rbf_oc_svm_test_time, 
                  'musk':musk_rbf_oc_svm_test_time, 
                  'bank':bank_rbf_oc_svm_test_time}
test_time_oc_svm_rbf_df = pd.DataFrame.from_dict(test_time_oc_svm_rbf, orient='index', columns = ['Testing time']).reset_index()

In [161]:
total_time_oc_svm_rbf = {'arrhythmia':arrhythmia_rbf_oc_svm_train_time + arrhythmia_rbf_oc_svm_test_time,
                       'cardio':cardio_rbf_oc_svm_train_time + cardio_rbf_oc_svm_test_time, 
                        'forestcover':forestcover_rbf_oc_svm_train_time + forestcover_rbf_oc_svm_test_time, 
                       'annthyroid':annthyroid_rbf_oc_svm_train_time + annthyroid_rbf_oc_svm_test_time,       
                        'creditcard': creditcard_rbf_oc_svm_train_time + creditcard_rbf_oc_svm_test_time, 
                       'mammography':mammography_rbf_oc_svm_train_time + mammography_rbf_oc_svm_test_time, 
                        'shuttle':shuttle_rbf_oc_svm_train_time + shuttle_rbf_oc_svm_test_time, 
                      'mnist':mnist_rbf_oc_svm_train_time + mnist_rbf_oc_svm_test_time, 
                  'vowels':vowels_rbf_oc_svm_train_time + vowels_rbf_oc_svm_test_time, 
                  'seismic':seismic_rbf_oc_svm_train_time + seismic_rbf_oc_svm_test_time, 
                  'musk':musk_rbf_oc_svm_train_time + musk_rbf_oc_svm_test_time, 
                  'bank':bank_rbf_oc_svm_train_time + bank_rbf_oc_svm_test_time}
total_time_oc_svm_rbf_df = pd.DataFrame.from_dict(total_time_oc_svm_rbf, orient='index', columns = ['Total time']).reset_index()

In [162]:
pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(f1_score_oc_svm_rbf_df, recall_oc_svm_rbf_df, how = 'inner'), 
                                    precision_oc_svm_rbf_df, how ='inner'),
         auc_oc_svm_rbf_df, how = 'inner'), auprc_oc_svm_rbf_df, how = 'inner'), training_time_oc_svm_rbf_df, how = 'inner'), 
         test_time_oc_svm_rbf_df, how = 'inner'),total_time_oc_svm_rbf_df, how = 'inner')

,index,F1 score,Recall,Precision,AUC,AUPRC,Training time,Testing time,Total time
0,arrhythmia,0.407080,0.348485,0.489362,0.762600,0.408949,0.015625,0.015625,0.031250
1,cardio,0.422222,0.431818,0.413043,0.856053,0.413341,0.031250,0.062500,0.093750
2,forestcover,0.134596,0.768111,0.073761,0.931267,0.128591,1130.015625,1677.437500,2807.453125
3,annthyroid,0.193291,0.226592,0.168524,0.618189,0.144098,0.437500,1.046875,1.484375
4,creditcard,0.029823,0.878049,0.015169,0.945401,0.386689,1682.750000,1827.734375,3510.484375
5,mammography,0.180223,0.434615,0.113682,0.634955,0.113142,1.203125,2.531250,3.734375
6,shuttle,0.546060,0.654230,0.468584,0.943491,0.688576,31.578125,47.859375,79.437500
7,mnist,0.313993,0.328571,0.300654,0.759431,0.261307,0.671875,1.562500,2.234375
8,vowels,0.164948,0.320000,0.111111,0.704651,0.164318,0.015625,0.046875,0.062500
9,seismic,0.148837,0.188235,0.123077,0.403962,0.064502,0.062500,0.140625,0.203125
